In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('oversample_train_df.csv')

In [ ]:
df = df.drop(['text_en', 'date'], axis=1)

In [ ]:
df.head(3)

In [ ]:
! pip install pyMorfologik

In [ ]:
# https://github.com/dmirecki/pyMorfologik

In [ ]:
! pip install stop_words

In [ ]:
from nltk.stem.snowball import SnowballStemmer
import re
from string import punctuation
import nltk
import spacy
from stop_words import get_stop_words
from pyMorfologik import Morfologik
from pyMorfologik.parsing import ListParser
import string

parser = ListParser()
stemmer = Morfologik()

stopwords_pl = get_stop_words("pl")

In [ ]:
def preprocess_text(text):
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    new_text = text.translate(translator)
    new_text = re.sub(r'\d+', '', new_text)
    new_text = re.sub(r'\s+', ' ', new_text)
    new_text = new_text.strip()
    new_text = new_text.lower()

    stems = stemmer.stem([new_text], parser)
    tokens = [(list(stems[i][1].keys())[0] if len(list(stems[i][1].keys())) > 0 else stems[i][0]) for i in range(len(stems))]

    filtered_tokens = [token for token in tokens if token not in stopwords_pl]
    filtered_tokens = [token for token in filtered_tokens if token!= '']
    processed_text = " ".join(filtered_tokens)

    return processed_text

In [ ]:
from pyMorfologik import Morfologik
from pyMorfologik.parsing import ListParser

parser = ListParser()
stemmer = Morfologik()
stems = stemmer.stem(['Ala ma kota'], parser)
stems

In [ ]:
x = list(stems[0][1].keys())[0] if len(list(stems[0][1].keys())) > 0 else stems[0][0]
x

In [ ]:
sample_text = df['text'][0]

In [ ]:
sample_text

In [ ]:
translator = str.maketrans("", "", punctuation)
new_text = sample_text.translate(translator)
new_text = re.sub(r'\d+', '', new_text)
new_text = re.sub(r'\s+', ' ', new_text)
new_text = new_text.strip()
new_text = new_text.lower()

In [ ]:
new_text

In [ ]:
stems = stemmer.stem([new_text], parser)

In [ ]:
stems

In [ ]:
len(stems)

In [ ]:
tokenized_text = [(list(stems[i][1].keys())[0] if len(list(stems[i][1].keys())) > 0 else stems[i][0]) for i in range(len(stems))]
tokenized_text


In [ ]:
preprocess_text(df['text'][0])

In [ ]:
prep_texts = [preprocess_text(df['text'][i]) for i in range(len(df))]
df['prep_text'] = prep_texts
df[['text', 'prep_text']]

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(df['prep_text'])
print(vectors.shape)

In [ ]:
print(vectors[0])

In [ ]:
vectorizer.get_feature_names_out()

In [ ]:
print(vectors[0].toarray())

In [ ]:
! pip install joblib

In [ ]:
import joblib

joblib.dump(vectorizer, 'tfidf_vectorizer_wibit.joblib')

loaded_vectorizer = joblib.load('tfidf_vectorizer_wibit.joblib')

In [ ]:
new_sample_text = "Jestem turystą, który uwielbia odwiedzać zamki i różnego rodzaju lochy, w szczególności sale tortur. Najchętniej jadam steki w renomowanych restauracjach"
prep_new_sample =  preprocess_text(new_sample_text)
print(prep_new_sample)

new_vector = loaded_vectorizer.transform([prep_new_sample])
print(new_vector)
print('')
new_vector = vectorizer.transform([prep_new_sample])
print(new_vector)

In [ ]:
df.to_csv('oversample_stemmed_train_df.csv', index=False)

In [ ]:
# https://radimrehurek.com/gensim/models/doc2vec.html

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

VECTOR_SIZE = 100
MIN_COUNTS = 3
WINDOW = 5
EPOCHS = 100

tokenized_docs = [df['prep_text'][i].split(' ') for i in range(len(df))]
tagged_data = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(tokenized_docs)]

doc2vec_model = Doc2Vec(vector_size=VECTOR_SIZE, window=WINDOW, min_count=MIN_COUNTS, epochs=EPOCHS, seed=2023)
doc2vec_model.build_vocab(tagged_data)
doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

In [ ]:
doc2vec_model.save('doc2vec_model_wibit')
loaded_doc2vec_model = Doc2Vec.load('doc2vec_model_wibit')

In [ ]:
new_sample_text = "Jestem turystą, który uwielbia odwiedzać zamki i różnego rodzaju lochy, w szczególności sale tortur. Najchętniej jadam steki w renomowanych restauracjach"
prep_new_sample =  preprocess_text(new_sample_text)
tokenized_new_sample = prep_new_sample.split(' ')

new_vector = doc2vec_model.infer_vector(tokenized_new_sample)
print(new_vector)

new_vector = loaded_doc2vec_model.infer_vector(tokenized_new_sample)
print(new_vector)